# Data from Transfermarkt

In [1]:
#https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
import pandas as pd
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_from_div(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the div with class 'responsive-table'
        div = soup.find('div', {'class': 'responsive-table'})
        
        if div:
            # Extract headers
            headers = div.find_all('th')
            header_texts = [header.get_text(strip=True) for header in headers]
            header_texts.append('Club')  # Adding 'Club' as a header
            
            # Extract contents
            rows = div.find_all('tr')
            for row in rows[1:]:  # Skipping the first row as it's the header row
                player_data = {}
                
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                
                # Checking if the row is complete (same length as headers)
                if len(cell_texts) == len(header_texts) - 1:
                    for index, header in enumerate(header_texts[:-1]):
                        player_data[header] = cell_texts[index]
                    
                    # Extracting the club name from the 'title' attribute
                    club_tag = cells[3].find('a')
                    if club_tag and 'title' in club_tag.attrs:
                        club_name = club_tag['title']
                        player_data['Club'] = club_name
                    
                    data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')
    
    return data

messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
result_messi = get_data_from_div(messi)

result_ronaldo = get_data_from_div(ronaldo)

# Convert to DataFrame
df_messi = pd.DataFrame(result_messi)
df_ronaldo = pd.DataFrame(result_ronaldo)

#print(df_messi)
#print(df_ronaldo)

In [3]:
print(df_ronaldo)

   Season Competition reihenfolge ASC, saison_id DESC, verein_id ASC  \
0             Total :                                                  
1   23/24                                           Saudi Pro League   
2   23/24                                       AFC Champions League   
3   22/23                                           Saudi Pro League   
4   22/23                                              Europa League   
..    ...         ...                                            ...   
63  04/05                                             Premier League   
64  03/04                                                     FA Cup   
65  03/04                                             Premier League   
66  02/03                                           Taça de Portugal   
67  02/03                                              Liga Portugal   

                 Club 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90 90+  \
0                         92     113     112   7     11

In [45]:
ballon_dor_df = pd.read_csv(r"BallonDOr.csv")
#print(ballon_dor_df["Year"]==2013)


['Cristiano Ronaldo' 'Lionel Messi' 'Franck Ribéry' 'Zlatan Ibrahimović'
 'Neymar' 'Andrés Iniesta' 'Robin van Persie' 'Arjen Robben' 'Gareth Bale'
 'Andrea Pirlo' 'Radamel Falcao' 'Yaya Touré' 'Robert Lewandowski'
 'Philipp Lahm' 'Xavi' 'Mesut Özil' 'Bastian Schweinsteiger'
 'Thomas Müller' 'Luis Suárez Díaz' 'Edinson Cavani' 'Thiago Silva'
 'Eden Hazard' 'Manuel Neuer']


In [5]:
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"




In [6]:
#print(ballon_dor_df["Year"])
ballon_2008_2018 = ballon_dor_df.loc[ballon_dor_df['Year'] >= 2008]

p_08_18 = ballon_2008_2018["Player"].unique()
print(p_08_18)

['Cristiano Ronaldo' 'Lionel Messi' 'Fernando Torres' 'Iker Casillas'
 'Xavi' 'Andrei Arshavin' 'David Villa' 'Kaká' 'Zlatan Ibrahimović'
 'Steven Gerrard' 'Marcos Senna' 'Emmanuel Adebayor' 'Wayne Rooney'
 'Sergio Agüero' 'Frank Lampard' 'Franck Ribéry' "Samuel Eto'o"
 'Gianluigi Buffon' 'Cesc Fàbregas' 'Michael Ballack' 'Sergio Ramos'
 'Nemanja Vidić' 'Didier Drogba' 'Edwin van der Sar' 'Ruud van Nistelrooy'
 'Andrés Iniesta' 'Edin Džeko' 'Ryan Giggs' 'Thierry Henry'
 'Luís Fabiano Clemente' 'Diego Forlán' 'Yoann Gourcuff'
 'Júlio César Soares Espíndola' 'Maicon' 'Diego Ribas da Cunha'
 'John Terry' 'Yaya Touré' 'Wesley Sneijder' 'Xabi Alonso' 'Carles Puyol'
 'Mesut Özil' 'Arjen Robben' 'Thomas Müller' 'Bastian Schweinsteiger'
 'Asamoah Gyan' 'Júlio César' 'Miroslav Klose' 'Philipp Lahm' 'Dani Alves'
 'Luis Suárez Díaz' 'Neymar' 'Éric Abidal' 'Karim Benzema' 'Nani'
 'Gerard Piqué' 'Radamel Falcao' 'Andrea Pirlo' 'Robin van Persie'
 'Manuel Neuer' 'Sergio Busquets' 'Mario Balotelli' '

In [12]:
print(p_08_18[8])

Zlatan Ibrahimović


In [13]:
#!pip install --upgrade Selenium
#!pip install --upgrade ChromeDriver

In [14]:
print(p_08_18[8].lower())

zlatan ibrahimović


In [15]:
import unicodedata

lower_list = []

for i in p_08_18:
    #j = i.split()
    #q = "-".join(j)
    j = i.lower()
    
    slug =  unicodedata.normalize('NFD', j).encode('ascii', 'ignore').decode('ascii')
    #print(slug)
    lower_list.append(slug)
    
    
print(lower_list)

['cristiano ronaldo', 'lionel messi', 'fernando torres', 'iker casillas', 'xavi', 'andrei arshavin', 'david villa', 'kaka', 'zlatan ibrahimovic', 'steven gerrard', 'marcos senna', 'emmanuel adebayor', 'wayne rooney', 'sergio aguero', 'frank lampard', 'franck ribery', "samuel eto'o", 'gianluigi buffon', 'cesc fabregas', 'michael ballack', 'sergio ramos', 'nemanja vidic', 'didier drogba', 'edwin van der sar', 'ruud van nistelrooy', 'andres iniesta', 'edin dzeko', 'ryan giggs', 'thierry henry', 'luis fabiano clemente', 'diego forlan', 'yoann gourcuff', 'julio cesar soares espindola', 'maicon', 'diego ribas da cunha', 'john terry', 'yaya toure', 'wesley sneijder', 'xabi alonso', 'carles puyol', 'mesut ozil', 'arjen robben', 'thomas muller', 'bastian schweinsteiger', 'asamoah gyan', 'julio cesar', 'miroslav klose', 'philipp lahm', 'dani alves', 'luis suarez diaz', 'neymar', 'eric abidal', 'karim benzema', 'nani', 'gerard pique', 'radamel falcao', 'andrea pirlo', 'robin van persie', 'manuel 

In [11]:
print(lower_list[8])

zlatan ibrahimovic


In [59]:
#Player name, player_id
#Manually scraped becuase it takes more than 180h+ for jupyter notebook/code and risk of memory error if interent fails 
my_dict = {"cristiano ronaldo": 8198, "lionel messi": 28003, "fernando torres": 7767, "iker casillas": 3979,"xavi": 7607, 
           "andrei arshavin":15378, "david villa": 7980, "kaka": 3366, "zlatan ibrahimovic": 3455, "steven gerrard":3109,
          "marcos senna": 7840, "emmanuel adebayor":8883, "wayne rooney": 3332, "sergio aguero": 26399,
           "frank lampard": 3169,"franck ribery":22068, "samuel eto'o": 4257, "gianluigi buffon": 5023, 
           "cesc fabregas":8806, "michael ballack": 63, "sergio ramos": 25557, "nemanja vidic": 19726, 
           "didier drogba": 3924, "edwin van der sar": 3516, "ruud van nistelrooy": 3407,"andres iniesta": 7600, 
           "edin dzeko":28396, "ryan giggs": 3406, "thierry henry": 3207, "luis fabiano clemente": 19802,
           "diego forlan": 3408, "yoann gourcuff":18912, "julio cesar soares espindola": 22412, "maicon": 18301,
           "diego ribas da cunha": 4248, "john terry": 3160, "yaya toure": 13091,"wesley sneijder": 4673, 
           "xabi alonso":7476, "carles puyol": 7594, "mesut ozil": 35664, "arjen robben":4360, "thomas muller": 58358,
          "bastian schweinsteiger": 2514, "asamoah gyan": 26802,"julio cesar": 22412, "miroslav klose": 10,
           "philipp lahm": 2219, "dani alves": 15951,"luis suarez diaz":44352,"neymar": 68290, "eric abidal": 5283, 
          "karim benzema": 18922, "nani": 33706, "gerard pique": 18944, "radamel falcao": 39152, "andrea pirlo": 5817,
          "robin van persie": 4380,"manuel neuer": 17259, "sergio busquets": 65230, "mario balotelli":45416, 
          "gareth bale": 39381, "robert lewandowski": 38253, "edinson cavani": 48280, "thiago silva":29241, 
          "eden hazard": 50202, "james rodriguez": 88103, "toni kroos": 31909, "angel di maria": 45320, 
           "diego costa": 44779, "mario gotze": 74842, "javier mascherano": 19981,"thibaut courtois": 108390, 
           "paul pogba": 122153, "alexis sanchez":40433, "arturo vidal": 37666, "kevin de bruyne": 88755,
           "ivan rakitic": 32467, "antoine griezmann": 125781, "riyad mahrez": 171424, "jamie vardy":197838,
           "pepe": 14132, "pierre-emerick aubameyang": 58864, "rui patricio":45026, "luka modric": 27992, 
           "dimitri payet":37647, "paulo dybala":206050, "diego godin": 54928, "gonzalo higuain":39153,"koke": 74229,
           "hugo lloris": 17965, "kylian mbappe": 342229, "n'golo kante":225083, "harry kane":132098,"isco":85288,
           "marcelo": 44501, "david de gea": 59377, "leonardo bonucci": 39983, "sadio mane": 200512, "jan oblak": 121483,
           "mats hummels": 39728, "philippe coutinho": 80444, "dries mertens": 56416, "mohamed salah":148455, 
           "raphael varane": 164770, "roberto firmino":131789, "alisson": 105470, "mario mandzukic":34572
          }

##Took 1h and 30 min

In [79]:
### All the following strings are formatted with player_id in the actual code 
# Details_stats = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    #Season, Competition Club, 15min intervals of scored goals, Ekstra time, Total amount of goals 
    
    
# Trophies = https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/34572
    # Kind of trophies, which year/season the specific kind of trophy was won in.


# National team = https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/34572/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1
    #Kind of match in which tournament, Club, Opponant, Date, Result, Goals, Assists, Own goals, 
    # Minute for what kind of card was given to the player, minutes played of the match

# Stats per tournament = https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/34572
    #Competition, Club name, appearances for the club, Goals, asissists, cards, Minutes Played
    #The aboed are mentioned for each type of legue (National legue, Domestic cups and International cups)

# Stats per club = https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/34572
    #Club name, appearances for the club, Goals, asissists, cards, Minutes Played, 
    #The above are mentioned for each league the player played in 


players = []
for year in range(2008,2019): 
    #print(year)
    m = ballon_dor_df.loc[ballon_dor_df["Year"]== year]
    
    for i in m["Player"].values: 
        slug =  unicodedata.normalize('NFD', i).encode('ascii', 'ignore').decode('ascii')
        lowered = slug.lower()
        try: 
            if lowered not in players:
                players.append(lowered)
                player_id = my_dict[lowered]
                print(lowered, player_id)
                new_url = f"https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
        
                #print(lowered, my_dict[lowered])
                result_player = get_data_from_div(new_url)
                df_player = pd.DataFrame(result_player)
                print(df_player)
               # break
                
        except: 
            pass
        break
        #result_zlatan = get_data_from_div(zlatan)

        # Convert to DataFrame
        #df_zlatan = pd.DataFrame(result_zlatan)
        

cristiano ronaldo 8198
   Season Competition reihenfolge ASC, saison_id DESC, verein_id ASC  \
0             Total :                                                  
1   23/24                                           Saudi Pro League   
2   23/24                                       AFC Champions League   
3   22/23                                           Saudi Pro League   
4   22/23                                              Europa League   
..    ...         ...                                            ...   
63  04/05                                             Premier League   
64  03/04                                                     FA Cup   
65  03/04                                             Premier League   
66  02/03                                           Taça de Portugal   
67  02/03                                              Liga Portugal   

                 Club 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90 90+  \
0                         92    

In [26]:
#Zlatan Ibrahimović
base_url = "https://www.transfermarkt.com/zlatan-ibrahimovic/profil/spieler/1"
p = base_url.split("/")[-4]
print(" ".join(p.split("-")))

zlatan ibrahimovic


In [27]:
99800/60/24/10

6.930555555555555

In [16]:
"""
from selenium import webdriver

base_url = "https://www.transfermarkt.com/silvio-adzic/profil/spieler/"
# Initialize the Chrome WebDriver
browser = webdriver.Chrome()


important_player_ids = {}

for player_id in range(0, 99998):
    url = base_url + str(player_id)
    try:
        browser.get(url)
        
        # Wait for a certain time (you can use other waits if needed)
        browser.implicitly_wait(0.1)

        # Get the current URL after the delay
        final_url = browser.current_url

        #print(final_url)
        p = final_url.split("/")[-4]
        string = " ".join(p.split("-"))
        #print(string, lower_list[8])
        
        if string in lower_list:
            print(string)
            # If found, perform actions or extract information here...
            
            # Stop the loop if Zlatan Ibrahimovic is found
            important_player_ids[p] = player_id
            
        
    except Exception as e:
        print("Exception occurred:", str(e))
    finally:
        # Close the browser after all iterations are done
        if player_id == 99997:  # Check if it's the last iteration
            browser.quit()
        # To avoid quitting the browser prematurely in other iterations

print(important_player_ids)
print(lower_list)
"""

'\nfrom selenium import webdriver\n\nbase_url = "https://www.transfermarkt.com/silvio-adzic/profil/spieler/"\n# Initialize the Chrome WebDriver\nbrowser = webdriver.Chrome()\n\n\nimportant_player_ids = {}\n\nfor player_id in range(0, 99998):\n    url = base_url + str(player_id)\n    try:\n        browser.get(url)\n        \n        # Wait for a certain time (you can use other waits if needed)\n        browser.implicitly_wait(0.1)\n\n        # Get the current URL after the delay\n        final_url = browser.current_url\n\n        #print(final_url)\n        p = final_url.split("/")[-4]\n        string = " ".join(p.split("-"))\n        #print(string, lower_list[8])\n        \n        if string in lower_list:\n            print(string)\n            # If found, perform actions or extract information here...\n            \n            # Stop the loop if Zlatan Ibrahimovic is found\n            important_player_ids[p] = player_id\n            \n        \n    except Exception as e:\n        pr

https://www.transfermarkt.com/zlatan-ibrahimovic/profil/spieler/3450


In [ ]:
zlatan = "https://www.transfermarkt.com/Zlatan-Ibrahimović/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"

result_zlatan = get_data_from_div(zlatan)

# Convert to DataFrame
df_zlatan = pd.DataFrame(result_zlatan)


neymar = "https://www.transfermarkt.com/Neymar/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"

result_neymar = get_data_from_div(neymar)

# Convert to DataFrame
df_neymar = pd.DataFrame(result_neymar)




In [ ]:
print(df_neymar)